<center><h1>DSCI 552: Machine Learning for Data Science</h1></center><br>
<center>Time Series Classification</center>

<h3>Homework 3</h3>
<p></p>
<b><span>Name: Sri Manvith Vaddeboyina</span></b><br>
<b><span>USC ID: 1231409457</span></b><br>
<b><span>Github username: srimanvithv</span></b>

## 1. Time Series Classification Part 1: Feature Creation/Extraction

### (a) Download Data

Package imports

In [1]:
import os
import re
import warnings
import numpy as np
import pandas as pd
from scipy.stats import bootstrap

warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)

Get the AReM Data Set

Download the AReM data from:<br>
https://archive.ics.uci.edu/ml/datasets/Activity+Recognition+system+based+on+Multisensor+data+fusion+\%28AReM\%29 <br><br>
The dataset contains 7 folders that represent seven types of activities. In
each folder, there are multiple files each of which represents an instant of a human
performing an activity. Each file containis 6 time series collected from activities of the same person, which are called avg_rss12, var_rss12, avg_rss13, var_rss13, vg_rss23, and ar_rss23. There are 88 instances in the dataset, each of which contains 6 time series and each time series has 480 consecutive values.

In [2]:
def load_and_preprocess_csv(file_path):
    if file_path == '../data/AReM/bending2/dataset4.csv':
        df = pd.read_csv(file_path, skiprows = 5,usecols = [1, 2, 3, 4, 5, 6],names=['avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23'], delim_whitespace = True) 

    else:
        df = pd.read_csv(file_path, skiprows = 4, usecols = [1, 2, 3, 4, 5, 6])

    return df

def load_files_in_directory(directory_path):
    files = {}
    for root, dirs, filenames in os.walk(directory_path):
        for filename in filenames:
            if filename.endswith('.csv'):
                file_path = os.path.join(root, filename)
                df = load_and_preprocess_csv(file_path)
                if df is not None:
                    df['label'] = file_path.split("/")[3]
                    files[file_path] = df
    return files

data_folder_path = '../data/AReM/'
files = load_files_in_directory(data_folder_path)

### (b) Test and Train Data

In [3]:
root = "../data/AReM"

train_files, test_files = [], []

train_dfs, test_dfs = {}, {}

files_ = [os.path.join(root_path, file_name) 
         for root_path, _, files in os.walk(root) 
         for file_name in files 
         if not file_name.endswith('.pdf')]

for i in files_:
    if 'bending1' in i or 'bending2' in i:
        if 'dataset1.csv' in i or 'dataset2.csv' in i:
            test_files.append(i)
        else:
            train_files.append(i)
    else:
        if 'dataset1.csv' in i or 'dataset2.csv' in i or 'dataset3.csv' in i:
            test_files.append(i)
        else:
            train_files.append(i)

for file_path in train_files:
    if file_path == '../data/AReM/bending2/dataset4.csv':
        df = pd.read_csv(file_path, skiprows = 5,usecols = [1, 2, 3, 4, 5, 6],names=['avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23'], delim_whitespace = True) 
    else:
        df = pd.read_csv(file_path, skiprows = 4, usecols = [1, 2, 3, 4, 5, 6])
    
    df['label'] = file_path.split("/")[3]
    train_dfs[file_path] = df

for file_path in test_files:
    if file_path == '../data/AReM/bending2/dataset4.csv':
        df = pd.read_csv(file_path, skiprows = 5,usecols = [1, 2, 3, 4, 5, 6],names=['avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23'], delim_whitespace = True) 
    else:
        df = pd.read_csv(file_path, skiprows = 4, usecols = [1, 2, 3, 4, 5, 6])  
    
    df['label'] = file_path.split("/")[3]
    test_dfs[file_path] = df
    
print('\033[1mTraining set size: ',len(train_files))
print('\033[1mTesting set size: ',len(test_files))

Training set size:  69
Testing set size:  19


### (c) Feature Extraction

#### i. Research

The time-domain features that are usually used in time series classification are: 
- <b>Mean</b>
- <b>Median</b>
- <b>Minimum</b>
- <b>Maximum</b>
- <b>Standard Deviation</b>
- <b>Variance</b>
- <b>Skewness</b>
- <b>Kurtosis</b>
- <b>Quantiles</b>
- <b>Range</b>
- <b>Interquartile Range (IQR)</b>

#### ii. Extraction

In [4]:
def extract(files_list):
    features = ['min', 'max', 'mean', '50%', 'std', '25%', '75%']

    time_series_statistical_features = ['min', 'max', 'mean', 'median', 'standard deviation', '1st quart', '3rd quart']

    data_list = []
    instance_column = list(range(1, len(files_list) + 1))
    
    labels = []
    for file_df in files_list.values():
        labels.append(file_df['label'][0])
        statistics = file_df.drop('label', axis=1).describe()
        feature_data = [statistics[feature][param] for feature in file_df.columns if feature != 'label' for param in features]
        data_list.append([len(data_list) + 1] + feature_data)
    
    columns = ['Instance'] + [f"{param}_{i+1}" for i in range(len(file_df.columns) - 1) for param in time_series_statistical_features]
    dataset = pd.DataFrame(data_list, columns=columns)
    dataset['Instance'] = instance_column
    dataset['label'] = labels
    return dataset

In [5]:
print("\033[1mComplete Dataset:")
complete_dataset_summary_label = extract(files)
complete_dataset_summary = extract(files).iloc[:,:-1]
display(complete_dataset_summary_label)

Complete Dataset:


,Instance,min_1,max_1,mean_1,median_1,standard deviation_1,1st quart_1,3rd quart_1,min_2,max_2,...,1st quart_5,3rd quart_5,min_6,max_6,mean_6,median_6,standard deviation_6,1st quart_6,3rd quart_6,label
0,1,36.25,48.00,43.969125,44.50,1.618364,43.3100,44.67,0.0,1.50,...,20.5000,23.7500,0.0,2.96,0.555313,0.490,0.487826,0.0000,0.8300,bending1
1,2,37.00,48.00,43.454958,43.25,1.386098,42.5000,45.00,0.0,1.58,...,22.2500,24.0000,0.0,5.26,0.679646,0.500,0.622534,0.4300,0.8700,bending1
2,3,33.00,47.75,42.179813,43.50,3.670666,39.1500,45.00,0.0,3.00,...,30.4575,36.3300,0.0,2.18,0.613521,0.500,0.524317,0.0000,1.0000,bending1
3,4,33.00,45.75,41.678063,41.75,2.243490,41.3300,42.75,0.0,2.83,...,28.4575,31.2500,0.0,1.79,0.383292,0.430,0.389164,0.0000,0.5000,bending1
4,5,37.25,45.00,40.624792,40.50,1.476967,39.2500,42.00,0.0,1.30,...,33.0000,36.0000,0.0,1.92,0.570583,0.430,0.582915,0.0000,1.3000,bending1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,84,18.50,44.25,35.752354,36.00,4.614802,33.0000,39.33,0.0,12.60,...,14.0000,18.0625,0.0,9.39,3.069667,2.770,1.748326,1.7975,4.0600,cycling
84,85,24.25,45.00,37.177042,36.25,3.581301,34.5000,40.25,0.0,8.58,...,17.9500,21.7500,0.0,9.34,2.921729,2.500,1.852600,1.5000,3.9000,cycling
85,86,23.33,43.50,36.244083,36.75,3.822016,33.4575,39.25,0.0,9.71,...,15.7500,21.0000,0.0,11.15,3.530500,3.110,1.963685,2.1700,4.6175,cycling
86,87,26.25,44.25,36.957458,36.29,3.434863,34.5000,40.25,0.0,8.64,...,14.0000,18.2500,0.0,8.34,2.934625,2.525,1.631380,1.6600,4.0300,cycling


In [6]:
print("\033[1mTrain Dataset:")
train_dataset_summary_label = extract(train_dfs)
train_dataset_summary = extract(train_dfs).iloc[:,:-1]
display(train_dataset_summary_label)

Train Dataset:


,Instance,min_1,max_1,mean_1,median_1,standard deviation_1,1st quart_1,3rd quart_1,min_2,max_2,...,1st quart_5,3rd quart_5,min_6,max_6,mean_6,median_6,standard deviation_6,1st quart_6,3rd quart_6,label
0,1,36.25,48.00,43.969125,44.50,1.618364,43.3100,44.67,0.0,1.50,...,20.5000,23.7500,0.0,2.96,0.555313,0.490,0.487826,0.0000,0.8300,bending1
1,2,37.00,48.00,43.454958,43.25,1.386098,42.5000,45.00,0.0,1.58,...,22.2500,24.0000,0.0,5.26,0.679646,0.500,0.622534,0.4300,0.8700,bending1
2,3,33.00,47.75,42.179813,43.50,3.670666,39.1500,45.00,0.0,3.00,...,30.4575,36.3300,0.0,2.18,0.613521,0.500,0.524317,0.0000,1.0000,bending1
3,4,33.00,45.75,41.678063,41.75,2.243490,41.3300,42.75,0.0,2.83,...,28.4575,31.2500,0.0,1.79,0.383292,0.430,0.389164,0.0000,0.5000,bending1
4,5,35.00,47.40,43.954500,44.33,1.558835,43.0000,45.00,0.0,1.70,...,35.3625,36.5000,0.0,1.79,0.493292,0.430,0.513506,0.0000,0.9400,bending1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,65,18.50,44.25,35.752354,36.00,4.614802,33.0000,39.33,0.0,12.60,...,14.0000,18.0625,0.0,9.39,3.069667,2.770,1.748326,1.7975,4.0600,cycling
65,66,24.25,45.00,37.177042,36.25,3.581301,34.5000,40.25,0.0,8.58,...,17.9500,21.7500,0.0,9.34,2.921729,2.500,1.852600,1.5000,3.9000,cycling
66,67,23.33,43.50,36.244083,36.75,3.822016,33.4575,39.25,0.0,9.71,...,15.7500,21.0000,0.0,11.15,3.530500,3.110,1.963685,2.1700,4.6175,cycling
67,68,26.25,44.25,36.957458,36.29,3.434863,34.5000,40.25,0.0,8.64,...,14.0000,18.2500,0.0,8.34,2.934625,2.525,1.631380,1.6600,4.0300,cycling


In [7]:
print("\033[1mTest Dataset:")
test_dataset_summary_label = extract(test_dfs)
test_dataset_summary = extract(test_dfs).iloc[:,:-1]
display(test_dataset_summary_label)

Test Dataset:


,Instance,min_1,max_1,mean_1,median_1,standard deviation_1,1st quart_1,3rd quart_1,min_2,max_2,...,1st quart_5,3rd quart_5,min_6,max_6,mean_6,median_6,standard deviation_6,1st quart_6,3rd quart_6,label
0,1,37.25,45.00,40.624792,40.500,1.476967,39.2500,42.0000,0.0,1.30,...,33.0000,36.0000,0.0,1.92,0.570583,0.430,0.582915,0.0000,1.300,bending1
1,2,38.00,45.67,42.812812,42.500,1.435550,42.0000,43.6700,0.0,1.22,...,32.0000,34.5000,0.0,3.11,0.571083,0.430,0.601010,0.0000,1.300,bending1
2,3,19.33,43.50,34.227771,35.500,4.889576,30.5000,37.7500,0.0,14.50,...,14.7500,18.6700,0.0,9.74,3.394125,3.100,1.792090,2.1050,4.425,walking
3,4,12.50,45.00,33.509729,34.125,4.850923,30.5000,36.7500,0.0,13.05,...,14.6275,18.7500,0.0,8.96,3.378479,3.085,1.787360,2.0600,4.440,walking
4,5,15.00,46.75,34.660583,35.000,5.315110,31.0000,38.2500,0.0,13.44,...,14.2500,18.5000,0.0,8.99,3.244396,3.000,1.630983,2.1200,4.240,walking
5,6,12.75,51.00,24.562958,24.250,3.737514,23.1875,26.5000,0.0,6.87,...,20.5000,27.0000,0.0,4.97,0.700188,0.500,0.693720,0.4300,0.870,bending2
6,7,0.00,42.75,27.464604,28.000,3.583582,25.5000,30.0000,0.0,7.76,...,15.0000,20.7500,0.0,6.76,1.122125,0.830,1.012342,0.4700,1.300,bending2
7,8,33.33,48.00,44.334729,45.000,2.476940,42.2500,46.5000,0.0,3.90,...,9.3300,17.7500,0.0,5.02,0.933000,0.830,0.673609,0.4700,1.250,standing
8,9,35.50,46.25,43.174938,43.670,1.989052,42.5000,44.5000,0.0,2.12,...,12.7500,16.5000,0.0,5.72,0.911979,0.830,0.666161,0.4700,1.220,standing
9,10,32.75,47.00,42.760563,44.500,3.398919,41.3300,45.3725,0.0,3.34,...,13.0000,18.5650,0.0,5.73,0.842271,0.710,0.722165,0.4300,1.090,standing


#### iii. Standard Deviation

In [8]:
std_complete_dataset = pd.DataFrame({'Feature': complete_dataset_summary.drop(columns=['Instance']).std().index, 'Standard Deviation': complete_dataset_summary.drop(columns=['Instance']).std().values})
print("\033[1mComplete Dataset:")
std_complete_dataset

Complete Dataset:


,Feature,Standard Deviation
0,min_1,9.569975
1,max_1,4.394362
2,mean_1,5.335718
3,median_1,5.440054
4,standard deviation_1,1.772153
5,1st quart_1,6.153590
6,3rd quart_1,5.138925
7,min_2,0.000000
8,max_2,5.062729
9,mean_2,1.574164


In [9]:
std_train_dataset = pd.DataFrame({'Feature': train_dataset_summary.drop(columns=['Instance']).std().index, 'Standard Deviation': train_dataset_summary.drop(columns=['Instance']).std().values})
print("\033[1mTrain Dataset:")
std_train_dataset

Train Dataset:


,Feature,Standard Deviation
0,min_1,8.794295
1,max_1,4.429182
2,mean_1,4.917717
3,median_1,4.956111
4,standard deviation_1,1.758629
5,1st quart_1,5.731262
6,3rd quart_1,4.783645
7,min_2,0.000000
8,max_2,5.147841
9,mean_2,1.600661


In [10]:
std_test_dataset = pd.DataFrame({'Feature': test_dataset_summary.drop(columns=['Instance']).std().index, 'Standard Deviation': test_dataset_summary.drop(columns=['Instance']).std().values})
print("\033[1mTest Dataset:")
std_test_dataset

Test Dataset:


,Feature,Standard Deviation
0,min_1,12.136206
1,max_1,4.379342
2,mean_1,6.790086
3,median_1,7.088085
4,standard deviation_1,1.869285
5,1st quart_1,7.673052
6,3rd quart_1,6.385720
7,min_2,0.000000
8,max_2,4.870395
9,mean_2,1.500529


In [11]:
confidence_interval = [
    [col, complete_dataset_summary[col].std(), bootstrap((complete_dataset_summary[col],), np.std, confidence_level=0.9, random_state=1, method='percentile').confidence_interval.low, bootstrap((complete_dataset_summary[col],), np.std, confidence_level=0.9, random_state=1, method='percentile').confidence_interval.high]
    for col in complete_dataset_summary.columns]

bootstrap_complete_dataset_summary = pd.DataFrame(confidence_interval, columns=['Feature Name', 'Standard Deviation', 'Lower Bound', 'Upper Bound'])[1:]
print("\033[1mComplete Dataset:")
bootstrap_complete_dataset_summary

Complete Dataset:


,Feature Name,Standard Deviation,Lower Bound,Upper Bound
1,min_1,9.569975,8.217348,10.704231
2,max_1,4.394362,3.306005,5.269063
3,mean_1,5.335718,4.675214,5.837115
4,median_1,5.440054,4.760294,5.964649
5,standard deviation_1,1.772153,1.562763,1.935570
6,1st quart_1,6.153590,5.542312,6.603819
7,3rd quart_1,5.138925,4.308947,5.806972
8,min_2,0.000000,0.000000,0.000000
9,max_2,5.062729,4.595134,5.377385
10,mean_2,1.574164,1.393100,1.698068


In [12]:
confidence_interval = [
    [col, train_dataset_summary[col].std(), bootstrap((train_dataset_summary[col],), np.std, confidence_level=0.9, random_state=1, method='percentile').confidence_interval.low, bootstrap((train_dataset_summary[col],), np.std, confidence_level=0.9, random_state=1, method='percentile').confidence_interval.high]
    for col in train_dataset_summary.columns]

bootstrap_train_dataset_summary = pd.DataFrame(confidence_interval, columns=['Feature Name', 'Standard Deviation', 'Lower Bound', 'Upper Bound'])[1:]
print("\033[1mTrain Dataset:")
bootstrap_train_dataset_summary

Train Dataset:


,Feature Name,Standard Deviation,Lower Bound,Upper Bound
1,min_1,8.794295,7.471305,9.958659
2,max_1,4.429182,3.257759,5.343441
3,mean_1,4.917717,4.284080,5.385967
4,median_1,4.956111,4.297972,5.443220
5,standard deviation_1,1.758629,1.531020,1.932402
6,1st quart_1,5.731262,5.108366,6.174618
7,3rd quart_1,4.783645,3.878226,5.492035
8,min_2,0.000000,0.000000,0.000000
9,max_2,5.147841,4.618632,5.484835
10,mean_2,1.600661,1.391740,1.726822


In [13]:
confidence_interval = [
    [col, test_dataset_summary[col].std(), bootstrap((test_dataset_summary[col],), np.std, confidence_level=0.9, random_state=1, method='percentile').confidence_interval.low, bootstrap((test_dataset_summary[col],), np.std, confidence_level=0.9, random_state=1, method='percentile').confidence_interval.high]
    for col in test_dataset_summary.columns]

bootstrap_test_dataset_summary = pd.DataFrame(confidence_interval, columns=['Feature Name', 'Standard Deviation', 'Lower Bound', 'Upper Bound'])[1:]
print("\033[1mTest Dataset:")
bootstrap_test_dataset_summary

Test Dataset:


,Feature Name,Standard Deviation,Lower Bound,Upper Bound
1,min_1,12.136206,8.457455,14.310378
2,max_1,4.379342,1.762636,6.264174
3,mean_1,6.790086,4.752908,7.786925
4,median_1,7.088085,5.060133,8.106922
5,standard deviation_1,1.869285,1.266126,2.234854
6,1st quart_1,7.673052,5.820522,8.566772
7,3rd quart_1,6.385720,4.202041,7.486927
8,min_2,0.000000,0.000000,0.000000
9,max_2,4.870395,3.590871,5.396045
10,mean_2,1.500529,0.972045,1.740835


#### iv. Select Features

1. **Max (Maximum Value):**
   - **Reason:** Max captures extreme values, highlighting distinct motion characteristics. Useful for identifying unique traits in bending vs. non-bending.

2. **Mean (Average Value):**
   - **Reason:** Mean provides a central tendency measure, representing typical behavior. Robust against outliers and helpful for understanding overall motion patterns.

3. **75th Percentile (Third Quartile):**
   - **Reason:** The 75th percentile considers the upper data range, revealing relatively high values without extreme outliers. This feature aids in distinguishing bending from non-bending activities by focusing on significant upper quartile differences.

# 

## 2. ISLR 3.7.4

I collect a set of data (n = 100 observations) containing a single predictor and a quantitative response. I then fit a linear regression model to the data, as well as a separate cubic regression, i.e. <br>
<center>Y = β<sub>0</sub> + β<sub>1</sub>X + β<sub>2</sub>$X^2$ + β<sub>3</sub>$X^3$ + ε<center>

### (a) Linear Train

<b>(a) Suppose that the true relationship between X and Y is linear, i.e. Y = β0 + β1X + ε. Consider the training residual sum of squares (RSS) for the linear regression, and also the training RSS for the cubic regression. Would we expect one to be lower than the other, would we expect them to be the same, or is there not enough information to tell? Justify your answer.</b>

<b>Answer:</b> In this scenario, the expectation is that the training RSS of the cubic regression model would be lower than that of the linear regression model. This expectation is based on the notion that more predictors in cubic regression can result in a better fit to the training data. However, this doesn't necessarily imply that cubic regression is a better model, as it may overfit the data due to its excessive complexity for a linear relationship. The lower training RSS in cubic regression is primarily due to its increased flexibility in capturing complex relationships.

### (b) Linear Test

<b>(b) Answer (a) using test rather than training RSS.<b>

<b>Answer:</b> In scenarios with more predictors and higher-order terms, cubic regression tends to overfit due to increased variance without a corresponding reduction in bias when the true underlying model is linear. Consequently, for test data, linear regression is expected to have a lower Residual Sum of Squares (RSS) compared to cubic regression, as it is more likely to provide relatively accurate regression results and mitigate the issues associated with overfitting.

### (c) Not Linear Train

<b>(c) Suppose that the true relationship between X and Y is not linear, but we don’t know how far it is from linear. Consider the training RSS for the linear regression, and also the training RSS for the cubic regression. Would we expect one to be lower than the other, would we expect them to be the same, or is there not enough information to tell? Justify your answer.</b>

<b>Answer:</b> Even in cases where the true underlying relationship between variables is unknown, cubic regression often offers superior flexibility when more predictors are involved. This increased flexibility allows cubic regression to capture complex patterns and variations in the data more effectively than linear regression. Consequently, during training, cubic regression tends to produce a lower Residual Sum of Squares (RSS) compared to linear regression. This is because higher model flexibility generally leads to a better fit to the training data, even when the true model is uncertain. Thus, the training RSS for cubic regression is expected to be smaller due to its capacity to adapt to intricate data patterns.

### (d) Not Linear Testing

<b>(d) Answer (c) using test rather than training RSS.</b>

<b>Answer:</b> The choice between linear and cubic regression's training RSS depends on the true underlying relationship between X and Y. If the relationship is approximately quadratic, either model could yield a lower RSS. However, when the true relationship deviates significantly from linearity, cubic regression is expected to outperform linear regression, highlighting the importance of aligning model choice with the underlying data dynamics.

<center><b>Thank You</b></center>